In [1]:
from blackops.utils.catalog import read_table
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import log_loss, f1_score

In [2]:
df = read_table(table_name="loan_data").toPandas()
display(df)

24/11/12 02:32:13 WARN Utils: Your hostname, dadiego resolves to a loopback address: 127.0.1.1; using 192.168.104.128 instead (on interface eth1)
24/11/12 02:32:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/dadiego/projects/esic-bigdata-iv-blackops/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/dadiego/.ivy2/cache
The jars for the packages stored in: /home/dadiego/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
io.delta#delta-sharing-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ba9072b9-2afe-435c-9d10-263b7dd1fb71;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found io.delta#delta-sharing-spark_2.12;3.2.0 in central
	found io.delta#delta-sharing-client_2.12;1.0.5 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found commons-logging#commons-logging;1.2 in central
	found commons-codec#commons-codec;1.11 in central
:: resolution report :: resolve 323ms :: artifacts dl 20ms
	:: modules in use:
	commons-codec#commons-codec;1.11 from central in [default]
	commons-logging#commons-log

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44995,27.0,male,Associate,47971.0,6,RENT,15000.0,MEDICAL,15.66,0.31,3.0,645,No,1
44996,37.0,female,Associate,65800.0,17,RENT,9000.0,HOMEIMPROVEMENT,14.07,0.14,11.0,621,No,1
44997,33.0,male,Associate,56942.0,7,RENT,2771.0,DEBTCONSOLIDATION,10.02,0.05,10.0,668,No,1
44998,29.0,male,Bachelor,33164.0,4,RENT,12000.0,EDUCATION,13.23,0.36,6.0,604,No,1


In [3]:
X = df.drop(columns="loan_status")
y = df.loan_status

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)

In [5]:
column_transformer = ColumnTransformer(
    transformers=[
        (
            "ohe",
            OneHotEncoder(drop="first", sparse_output=False),
            [
                "person_gender",
                "person_education",
                "person_home_ownership",
                "loan_intent",
                "previous_loan_defaults_on_file",
            ],
        )
    ],
    remainder="passthrough",
    verbose_feature_names_out=False,
)
pipe = Pipeline(
    steps=[
        ("encoding", column_transformer),
        ("scaler", StandardScaler()),
        ("clf", CatBoostClassifier()),
    ]
)#.set_output(transform="pandas")

In [6]:
pipe.fit(X_train, y_train)

Learning rate set to 0.044115
0:	learn: 0.5983832	total: 56.1ms	remaining: 56.1s
1:	learn: 0.5192312	total: 60.7ms	remaining: 30.3s
2:	learn: 0.4590137	total: 63.3ms	remaining: 21s
3:	learn: 0.4122340	total: 65.8ms	remaining: 16.4s
4:	learn: 0.3735166	total: 68.3ms	remaining: 13.6s
5:	learn: 0.3458701	total: 70.7ms	remaining: 11.7s
6:	learn: 0.3222582	total: 73.1ms	remaining: 10.4s
7:	learn: 0.3032355	total: 78ms	remaining: 9.68s
8:	learn: 0.2893599	total: 82.2ms	remaining: 9.05s
9:	learn: 0.2765046	total: 84.5ms	remaining: 8.37s
10:	learn: 0.2652544	total: 86.9ms	remaining: 7.81s
11:	learn: 0.2569821	total: 90.2ms	remaining: 7.43s
12:	learn: 0.2496143	total: 94.6ms	remaining: 7.18s
13:	learn: 0.2436718	total: 97.4ms	remaining: 6.86s
14:	learn: 0.2371964	total: 99.7ms	remaining: 6.55s
15:	learn: 0.2332247	total: 102ms	remaining: 6.28s
16:	learn: 0.2296163	total: 105ms	remaining: 6.04s
17:	learn: 0.2257674	total: 107ms	remaining: 5.84s
18:	learn: 0.2229661	total: 111ms	remaining: 5.74s


/home/dadiego/projects/esic-bigdata-iv-blackops/.venv/lib/python3.11/site-packages/sklearn/compose/_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('encoding',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['person_gender',
                                                   'person_education',
                                                   'person_home_ownership',
                                                   'loan_intent',
                                                   'previous_loan_defaults_on_file'])],
                                   verbose_feature_names_out=False)),
                ('scaler', StandardScaler()),
                ('clf',
                 <catboost.core.CatBoostClassifier object at 0x7fdac51cb190>)])

In [7]:
y_preds = pipe.predict_proba(X_test)[:, 1]

In [8]:
log_loss(y_true=y_test, y_pred=y_preds)

0.15496215900020888

In [11]:
f1_score(y_true=y_test, y_pred=pipe.predict(X_test))

0.8370015948963317